In [58]:
import re
import ast
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

In [31]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [32]:
out_df = pd.read_json(f"../data/annotated/mfc_annotated_Mistral.json", lines=True)
len(out_df)
frame_map = json.load(open("../data/raw/mfc_v4.0/codes.json",'r'))

In [41]:
multiple_labels_df = pd.read_csv("../data/processed/mfc_multi_label.csv")
multiple_labels_df = multiple_labels_df.dropna()

In [34]:
clean_preds = []
for ind, pred in enumerate(out_df['predictions']):
    pred = pred.replace("\n","")
    reasoning_begins = [m.start()-1 for m in re.finditer('"reasoning"', pred)]
    reasoning_ends = [pred[loc:].find("}") for loc in reasoning_begins]
    frame_annos = []
    prev = 0
    try:
        for i, (b,e) in enumerate(zip(reasoning_begins, reasoning_ends)):
            if i == 0:
                try:
                    frame_json = json.loads(pred[1:b-1]+"}")
                    frame_json['reasoning'] = pred[b+15:b+e-1]
                    frame_annos.append(frame_json)
                except:
                    continue
            else:
                try:
                    frame_json = json.loads("{" + pred[prev+1:b-1] + "}")
                    frame_json['reasoning'] = pred[b+15:b+e-1]
                    frame_annos.append(frame_json)
                except:
                    continue
            if i < len(reasoning_begins)-1:
                gap = pred[b+e:].index("{")
                prev = b+e+gap
    except:
        pass
    clean_preds.append(frame_annos)

In [35]:
out_df['clean_preds'] = clean_preds
out_df['pred_frames'] = out_df['clean_preds'].apply(lambda x: [frame_map[str(frame_json['frame_id'])+'.0'] if str(frame_json['frame_id'])+'.0' in frame_map.keys() else None for frame_json in x ])

In [40]:
print(len(multiple_labels_df.dropna()), len(out_df))

31210 31210


In [59]:
out_df['multiple_labels'] = [ast.literal_eval(x) for x in multiple_labels_df['multiple_labels_names'].values]

In [56]:
from pdb import set_trace

In [61]:
correct = []
for ind, row in out_df[['pred_frames','multiple_labels']].iterrows():
    flag = 0
    for label in row['pred_frames']:
        if label in row['multiple_labels']:
            flag = 1
    if flag:
        correct.append(True)
    else:
        correct.append(False)

In [63]:
correct = np.array(correct)
np.unique(correct, return_counts=True)

(array([False,  True]), array([ 7045, 24165]))

In [64]:
# Multiple prediction, multi-label accuracy
sum(correct == True)/len(out_df)

0.7742710669657161

In [ ]:
out_df['correct'] = correct
out_df[out_df['correct'] == 0].sample(100)[['text', 'label', 'pred_frames']]